In [26]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2,pywrapcp

class tsp_calc(object):
    def __init__(self, city_names=None):
        # 城市名称
        self.df = pd.read_excel('./cities.xlsx')
        self.all_city = self.df['name'].values
        if city_names is not None:
            self.city_names = city_names
            self.df = self.df[self.df['name'].isin(city_names)]
        else:
            self.city_names = self.all_city
            
    def create_data_model_tsp(self):
        data = {}
        temp = pd.read_excel('./distance.xlsx', index_col=0)
        # city_names筛选
        temp = temp[(temp.index.isin(self.city_names))][self.city_names]
        print(temp)
        data['distance_matrix'] = temp.values/1000
        
        data['num_vehicles'] = 1 
        data['depot'] = 0 
        return data
    
    # 结果
    def get_solution_tsp(self,manager, routing, solution):
        print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
        index = routing.Start(0)
        route = []
        route_distance = 0
        while not routing.IsEnd(index):
            index_show = manager.IndexToNode(index)
            # 添加route
            route.append(index_show)
            previous_index = index
            # 下一个节点
            index = solution.Value(routing.NextVar(index))
            # 
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        return route,route_distance
    
    def work_tsp(self):
        # step1，初始三个参数字典
        data = self.create_data_model_tsp()
        # step2，tsp_size（城市）, num_vehicles（车）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # step3，.
        routing = pywrapcp.RoutingModel(manager)

        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # 输出
        route, route_distance = self.get_solution_tsp(manager,routing,solution)
        return route,route_distance

In [20]:
model = tsp_calc()
route, route_distance = model.work_tsp()
print(route)
print(route_distance)

总行驶里程: 19799 公里
[0, 6, 22, 21, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 10, 9, 8, 15, 13, 28, 29, 14, 20, 17, 18, 19, 7, 5, 16, 1, 4, 2, 3]
19799


In [27]:
city_names = ['北京', '天津', '南京']
model = tsp_calc(city_names = city_names)
model.create_data_model_tsp()
route, route_distance = model.work_tsp()
print(route)
print(route_distance)

         北京      天津       南京
北京        0  122476  1002497
天津   122476       0   935319
南京  1002497  935319        0
         北京      天津       南京
北京        0  122476  1002497
天津   122476       0   935319
南京  1002497  935319        0
总行驶里程: 2059 公里
[0, 1, 2]
2059
